In [14]:
import os
import convokit
from convokit import Corpus, Toxicity

corpus = convokit.Corpus(filename="datasets/reddit_coarse_discourse-corpus/corpus")




In [3]:
import os
os.listdir()





['Test_transformer.ipynb',
 'coarse_discourse_dump_reddit_with_votes.json',
 'Corpus_conversion.ipynb',
 '.DS_Store',
 'tox_dictionary.py',
 'corpus',
 '__pycache__',
 'fetch_toxicity.ipynb',
 'A1E_Upvote_Toxicity_Transformer.ipynb',
 '.ipynb_checkpoints',
 'toxicity_dictionary.json',
 'readme.rst']

In [5]:
os.chdir('../..')

In [6]:
os.listdir()

['LICENSE.md',
 'convokit',
 '.DS_Store',
 'requirements.txt',
 'Makefile',
 'website',
 'datasets',
 'tests',
 'README.md',
 'setup.py',
 '.gitignore',
 'CONTRIBUTING.md',
 'examples',
 'doc',
 '.git']

In [9]:
corpus.print_summary_stats()

Number of Users: 63573
Number of Utterances: 115827
Number of Conversations: 9483


In [15]:
tox = Toxicity()
tox.transform(corpus)

In [64]:
for convo in corpus.iter_conversations():
    convo_scores = 0
    count = 0

    for utt in convo.iter_utterances():        
        '''
            rerunning this takes over a day for our 110k+ comments since it uses an api with limited query rate, 
            we'll load them from toxicity_dictionary.json that was pre-fetched,
            for others using our transformer, please run self.get_toxicity over the utterances on their corpus.
        '''

        #utt_score = self._get_toxicity(utt.text)


        utt_score = toxicity_scores[utt.id]
        
        convo_scores+=utt_score
        count+=1

        utt.add_meta('toxicity', utt_score)

    convo.add_meta('averagetoxicity', convo_scores/count)

In [39]:
list(corpus.iter_utterances())[0].meta['toxicity']

0.21341012